## Imports

In [2]:
import pandas as pd
import numpy as np
import random

## readFile

In [3]:
path = r'example.xlsm'
data = pd.read_excel(path, sheet_name='Real Model', engine='openpyxl')
campus = pd.read_excel(path, sheet_name='Nombre campus', header=3, index_col=0, engine='openpyxl')
campus = campus.loc[:, ['Liste']]
data.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,NaN,taille établissement,500,NaN,NaN,taille établissement,1436,NaN,NaN,taille établissement,...,1438,NaN,NaN,taille établissement,1071,NaN,NaN,taille établissement,1462,NaN
1,NaN,espérance %,0.2,NaN,NaN,espérance %,0.15,NaN,NaN,espérance %,...,0.15,NaN,NaN,espérance %,0.15,NaN,NaN,espérance %,0.15,NaN
2,NaN,espérance,100,NaN,NaN,espérance,215.4,NaN,NaN,espérance,...,215.7,NaN,NaN,espérance,160.65,NaN,NaN,espérance,219.3,NaN
3,NaN,écart-type %,0.04,NaN,NaN,écart-type %,0.04,NaN,NaN,écart-type %,...,0.04,NaN,NaN,écart-type %,0.04,NaN,NaN,écart-type %,0.04,NaN
4,NaN,écart-type,20,NaN,NaN,écart-type,57.44,NaN,NaN,écart-type,...,57.52,NaN,NaN,écart-type,42.84,NaN,NaN,écart-type,58.48,NaN
5,NaN,écart-max,60,NaN,NaN,écart-max,172.32,NaN,NaN,écart-max,...,172.56,NaN,NaN,écart-max,128.52,NaN,NaN,écart-max,175.44,NaN
6,NaN,amplitude,120,NaN,NaN,amplitude,344.64,NaN,NaN,amplitude,...,345.12,NaN,NaN,amplitude,257.04,NaN,NaN,amplitude,350.88,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## preProcessing

In [4]:
# V1
# creation d'un dataframe qui contient la colonne date + ses noms de colonne
# df = pd.DataFrame(columns = campus['Liste'])
# frame = {'date': data.iloc[15:1111, 1]}
# date = pd.DataFrame(frame)
# date.reset_index(inplace=True)
# date = pd.DataFrame(date.iloc[:,1])
# date

# V2
# creation d'un dataframe qui ne contient que la colonne date
frame = {'date': data.iloc[13:, 0]}
df = pd.DataFrame(frame)
df.reset_index(inplace=True)
df = pd.DataFrame(df.iloc[:,1])



#### Définition des variables permettant de créer le DataFrame 'df':
* nCampus: nombre de campus à étudier
* cursorCampus: curseur désignant le campus en cours de traitement
* colOffset: nombre de colonnes entre 2 campus dans le fichier source
* indexFirstCol: position de la premiere colonne à lire dans le fichier source
* indexStart: index de la premiere ligne de donnée dans le fichier source
* indexStop: index de la derniere ligne de donnée dans le fichier source

In [5]:
NUMBER_OF_CAMPUS = len(campus)
COLUMN_OFFSET = 4
INDEX_FIRST_COLUMN = 3
INDEX_START = 13
INDEX_STOP = len(data.index)-1

cursorCampus = 0

In [6]:
campus.iloc[17,0]

'Campus 10'

#### Création du DataFrame 'df':

1. Traitement de la première colonne
2. Ré-itération de l'opération grâce à une boucle sur le nombre de campus

In [7]:
# V2
intermediaire = df
frame = {campus.iloc[cursorCampus,0]: data.iloc[INDEX_START:INDEX_STOP+1, INDEX_FIRST_COLUMN]}
buffer = pd.DataFrame(frame)
buffer.reset_index(inplace=True)
buffer = pd.DataFrame(buffer.iloc[:,1])
# buffer.loc[buffer[campus.iloc[cursorCampus,0]] < 0] = 0

intermediaire[campus.iloc[cursorCampus,0]]=buffer.iloc[:,0]
intermediaire.loc[~(intermediaire[campus.iloc[0,0]] > 0), campus.iloc[0,0]] = 0
cursorCampus += 1

for i in range(1, NUMBER_OF_CAMPUS):
    frame = {campus.iloc[cursorCampus,0]: data.iloc[INDEX_START:INDEX_STOP+1, INDEX_FIRST_COLUMN + i*COLUMN_OFFSET]}
    buffer = pd.DataFrame(frame)
    buffer.reset_index(inplace=True)
    buffer = pd.DataFrame(buffer.iloc[:,1])
    intermediaire[campus.iloc[cursorCampus,0]]=buffer.iloc[:,0]
    intermediaire.loc[~(intermediaire[campus.iloc[cursorCampus,0]] > 0), campus.iloc[cursorCampus,0]] = 0
    cursorCampus += 1

df = intermediaire
cursorCampus = 0

In [8]:
df.head(50)

,date,G. Charpak,Mines ICM,Mines Albi,Telecom Paris,Telecom sud Paris,IMT Business school,IMT Atlantique,IMT Douai,Campus 1,Campus 2,Campus 3,Campus 4,Campus 5,Campus 6,Campus 7,Campus 8,Campus 9,Campus 10
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-01-02 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-01-03 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-01-04 00:00:00,83,246,178,192,113,113,249,365,191,111,175,154,153,132,119,210,151,210
4,2022-01-05 00:00:00,100,184,131,234,123,86,318,329,190,184,166,173,155,139,198,112,177,160
5,2022-01-06 00:00:00,73,191,106,227,54,284,223,269,143,230,203,169,157,138,44,119,82,142
6,2022-01-07 00:00:00,86,165,114,222,85,215,281,238,182,240,133,151,135,143,112,124,123,185
7,2022-01-08 00:00:00,89,166,123,236,145,121,321,330,141,253,199,186,223,184,128,222,110,173
8,2022-01-09 00:00:00,54,298,101,216,106,133,75,270,150,169,178,162,176,111,112,170,147,95
9,2022-01-10 00:00:00,53,88,161,96,125,67,266,267,133,195,161,189,169,160,169,195,183,185


In [9]:
intermediaire.loc[~(intermediaire[campus.iloc[0,0]] > 0), campus.iloc[0,0]]

0      0
1      0
2      0
36     0
37     0
      ..
360    0
361    0
362    0
363    0
364    0
Name: G. Charpak, Length: 100, dtype: object

## Déclaration des fournisseurs

#### Ajout des données 'taille' & 'esperance' pour chaque campus

In [11]:
nb = pd.read_excel(path, sheet_name='Nombre campus', header=3, index_col=0, engine='openpyxl')
campus['Taille'] = nb.loc[:, ['tailletot (étudiants + chercheurs)']]
campus['Esperance'] = nb.loc[:, ['esperance']]
campus

,Liste,Taille,Esperance
1,G. Charpak,500,100
2,Mines ICM,1436,215
3,Mines Albi,1000,150
4,Telecom Paris,1529,229
5,Telecom sud Paris,925,139
6,IMT Business school,1500,225
7,IMT Atlantique,1800,270
8,IMT Douai,2180,327
9,Campus 1,1136,170
10,Campus 2,1547,232


#### Calcul et ajout au df 'Campus' du nombre de fournisseurs par campus en fonction du nombre de fournisseur par tranche de consommateur (paramètre: producerScope)

In [449]:
producerScope = 20
supplierList = []

for i in range(0, len(campus)):
    n = campus.iloc[i,2]//producerScope
    supplierList.append(n)
supplierList
campus['N. Producteurs']=supplierList
campus

,Liste,Taille,Esperance,N. Producteurs
1,G. Charpak,500,100,5
2,Mines ICM,1436,215,10
3,Mines Albi,1000,150,7
4,Telecom Paris,1529,229,11
5,Telecom sud Paris,925,139,6
6,IMT Business school,1500,225,11
7,IMT Atlantique,1800,270,13
8,IMT Douai,2180,327,16
9,Campus 1,962,144,7
10,Campus 2,1157,174,8


#### Création du df 'producers'

In [450]:
tabCamp = []
tabProd = []

for name in campus['Liste']:
    val = campus.loc[campus['Liste'] == name, 'N. Producteurs'].iloc[0]
    for n in range(0,val):
        char = "Prod_" + str(n)
        tabCamp.append(name)
        tabProd.append(char)

producers = pd.DataFrame(tabCamp, columns = ['Id_Campus'])
producers['Id_Prod'] = pd.DataFrame(tabProd)
producers

,Id_Campus,Id_Prod
0,G. Charpak,Prod_0
1,G. Charpak,Prod_1
2,G. Charpak,Prod_2
3,G. Charpak,Prod_3
4,G. Charpak,Prod_4
...,...,...
160,Campus 10,Prod_2
161,Campus 10,Prod_3
162,Campus 10,Prod_4
163,Campus 10,Prod_5


## Répartition aléatoire des commandes auprès des fournisseurs

#### Répartition uniforme: p(fournisseur) = 1/n

In [451]:
#args: dfCampus et dfData formaté
# def uniformProducerDistribution(intermediaire, producers):
    

In [452]:
iterator = intermediaire.columns.values.tolist()
iterator.remove('date')
orderList = {}

for camp in iterator:
    # Caclul des paramètres de chaque campus
    n = producers['Id_Campus'].str.count(camp).sum()
    p = 1/n
    buffer = pd.DataFrame({'date' : intermediaire['date']})
    prod = producers.loc[producers['Id_Campus'] == camp, 'Id_Prod']
    orders = []

    # Calcul du nombre de commandes pour chaque jour de l'année 
    # et pour chaque producteur suivant une loi uniforme
    for i in range(len(intermediaire[camp])):
        consumers = round(intermediaire[camp][i]*p)
        orders.append(consumers)
    
    # Pour chaque jour et chaque campus, ajout de la quantité de commande pour 
    for element in prod:
        buffer[element]= orders
    
    orderList[camp] = buffer
        
# orderList[camp] = orders
# orders

orderList['IMT Douai']

,date,Prod_0,Prod_1,Prod_2,Prod_3,Prod_4,Prod_5,Prod_6,Prod_7,Prod_8,Prod_9,Prod_10,Prod_11,Prod_12,Prod_13,Prod_14,Prod_15
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-01-02 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-01-03 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-01-04 00:00:00,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
4,2022-01-05 00:00:00,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2022-12-28 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2022-12-29 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
363,2022-12-30 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Pour chaque fournisseur, traduction du nombre de commandes en quantité de commande

#### Création des paniers repas et de leur valeur en terme de points d'encombrement

In [453]:
# meal = {'panier1': 1,
#          'panier2': 3,
#          'panier3': 6}

meal = [1, 2, 3, 4, 5, 6]
#ajouter prix
#

In [454]:
amount = 0
for i in range(0, 100):
    amount += random.choice(meal)
    print(amount)
#     print(random.choice(meal))

4
10
12
15
17
21
26
27
28
30
33
34
40
41
44
49
52
58
61
63
69
74
75
79
80
85
87
90
96
100
101
104
110
111
114
120
122
127
131
135
136
142
148
149
155
160
165
167
168
174
180
181
187
192
194
198
200
206
209
215
221
222
228
232
234
237
239
244
247
251
257
258
261
265
266
268
271
277
279
285
286
291
292
295
300
303
307
309
310
313
317
322
326
330
336
341
346
347
349
350


In [455]:
nb.loc[:, ['esperance']]

,esperance
1,100
2,215
3,150
4,229
5,139
6,225
7,270
8,327
9,144
10,174


In [456]:
orderList['IMT Douai']

,date,Prod_0,Prod_1,Prod_2,Prod_3,Prod_4,Prod_5,Prod_6,Prod_7,Prod_8,Prod_9,Prod_10,Prod_11,Prod_12,Prod_13,Prod_14,Prod_15
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-01-02 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-01-03 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-01-04 00:00:00,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
4,2022-01-05 00:00:00,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2022-12-28 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2022-12-29 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
363,2022-12-30 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [461]:
iterator = []
finalList = orderList.copy()

for camp in finalList:
    # campName est le campus courant sous forme de string
#     print(camp)
    
    iterator = finalList[camp].columns.values.tolist()
    iterator.remove('date')
    # iterator est la liste des producteurs pour chaque campus
    
    for prod in iterator:
        
        for day in finalList[camp]['date']:
            amount = 0
            if not (finalList[camp].loc[finalList[camp]['date'] == day, prod].iloc[0] ==0):
                for consummer in range (0, finalList[camp].loc[finalList[camp]['date'] == day, prod].iloc[0]):
                    amount += random.choice(meal)
                print(camp, amount)
                finalList[camp].loc[finalList[camp]['date'] == day, prod]=amount
                amount = 0
                


G. Charpak 166
G. Charpak 260
G. Charpak 198
G. Charpak 123
G. Charpak 74
G. Charpak 196
G. Charpak 156
G. Charpak 250
G. Charpak 81
G. Charpak 86
G. Charpak 247
G. Charpak 249
G. Charpak 176
G. Charpak 352
G. Charpak 174
G. Charpak 210
G. Charpak 146
G. Charpak 182
G. Charpak 172
G. Charpak 297
G. Charpak 167
G. Charpak 246
G. Charpak 180
G. Charpak 115
G. Charpak 170
G. Charpak 189
G. Charpak 250
G. Charpak 176
G. Charpak 297
G. Charpak 249
G. Charpak 274
G. Charpak 274
G. Charpak 271
G. Charpak 228
G. Charpak 262
G. Charpak 274
G. Charpak 181
G. Charpak 341
G. Charpak 181
G. Charpak 306
G. Charpak 306
G. Charpak 235
G. Charpak 271
G. Charpak 306
G. Charpak 242
G. Charpak 323
G. Charpak 282
G. Charpak 129
G. Charpak 281
G. Charpak 277
G. Charpak 237
G. Charpak 186
G. Charpak 184
G. Charpak 169
G. Charpak 270
G. Charpak 262
G. Charpak 292
G. Charpak 226
G. Charpak 271
G. Charpak 291
G. Charpak 157
G. Charpak 147
G. Charpak 303
G. Charpak 285
G. Charpak 176
G. Charpak 191
G. Charpak 25

G. Charpak 150
G. Charpak 144
G. Charpak 207
G. Charpak 191
G. Charpak 274
G. Charpak 137
G. Charpak 177
G. Charpak 247
G. Charpak 190
G. Charpak 269
G. Charpak 67
G. Charpak 122
G. Charpak 136
G. Charpak 277
G. Charpak 188
G. Charpak 131
G. Charpak 201
G. Charpak 190
G. Charpak 161
G. Charpak 281
G. Charpak 169
G. Charpak 209
G. Charpak 145
G. Charpak 175
G. Charpak 164
G. Charpak 228
G. Charpak 236
G. Charpak 292
G. Charpak 193
G. Charpak 109
G. Charpak 141
G. Charpak 108
G. Charpak 167
G. Charpak 176
G. Charpak 273
G. Charpak 173
G. Charpak 172
G. Charpak 262
G. Charpak 210
G. Charpak 210
G. Charpak 135
G. Charpak 92
G. Charpak 192
G. Charpak 245
G. Charpak 118
G. Charpak 159
G. Charpak 168
G. Charpak 131
G. Charpak 174
G. Charpak 212
G. Charpak 221
G. Charpak 158
G. Charpak 114
G. Charpak 261
G. Charpak 185
G. Charpak 206
G. Charpak 154
G. Charpak 227
G. Charpak 194
G. Charpak 168
G. Charpak 204
G. Charpak 233
G. Charpak 163
G. Charpak 187
G. Charpak 184
G. Charpak 144
G. Charpak 1

G. Charpak 258
G. Charpak 370
G. Charpak 195
G. Charpak 206
G. Charpak 249
G. Charpak 278
G. Charpak 271
G. Charpak 284
G. Charpak 218
G. Charpak 254
G. Charpak 270
G. Charpak 248
G. Charpak 260
G. Charpak 290
G. Charpak 233
G. Charpak 161
G. Charpak 158
G. Charpak 217
G. Charpak 276
G. Charpak 228
G. Charpak 280
G. Charpak 213
G. Charpak 280
G. Charpak 333
G. Charpak 221
G. Charpak 267
G. Charpak 268
G. Charpak 157
G. Charpak 196
G. Charpak 220
G. Charpak 316
G. Charpak 231
G. Charpak 191
G. Charpak 214
G. Charpak 272
G. Charpak 317
G. Charpak 243
G. Charpak 290
G. Charpak 253
G. Charpak 220
G. Charpak 225
G. Charpak 241
G. Charpak 262
G. Charpak 339
G. Charpak 209
G. Charpak 279
G. Charpak 265
G. Charpak 222
G. Charpak 298
G. Charpak 244
G. Charpak 264
G. Charpak 249
G. Charpak 284
G. Charpak 178
G. Charpak 176
G. Charpak 348
G. Charpak 315
G. Charpak 245
G. Charpak 264
G. Charpak 198
G. Charpak 303
G. Charpak 284
G. Charpak 171
G. Charpak 237
G. Charpak 273
G. Charpak 244
G. Charpak

Mines ICM 213
Mines ICM 182
Mines ICM 310
Mines ICM 260
Mines ICM 408
Mines ICM 237
Mines ICM 347
Mines ICM 248
Mines ICM 210
Mines ICM 203
Mines ICM 187
Mines ICM 342
Mines ICM 292
Mines ICM 153
Mines ICM 158
Mines ICM 241
Mines ICM 397
Mines ICM 224
Mines ICM 168
Mines ICM 197
Mines ICM 240
Mines ICM 251
Mines ICM 251
Mines ICM 282
Mines ICM 337
Mines ICM 231
Mines ICM 284
Mines ICM 280
Mines ICM 278
Mines ICM 304
Mines ICM 158
Mines ICM 247
Mines ICM 270
Mines ICM 286
Mines ICM 241
Mines ICM 352
Mines ICM 283
Mines ICM 237
Mines ICM 155
Mines ICM 188
Mines ICM 187
Mines ICM 169
Mines ICM 197
Mines ICM 231
Mines ICM 191
Mines ICM 207
Mines ICM 148
Mines ICM 219
Mines ICM 131
Mines ICM 174
Mines ICM 206
Mines ICM 207
Mines ICM 241
Mines ICM 232
Mines ICM 254
Mines ICM 124
Mines ICM 190
Mines ICM 196
Mines ICM 173
Mines ICM 240
Mines ICM 175
Mines ICM 126
Mines ICM 195
Mines ICM 203
Mines ICM 192
Mines ICM 169
Mines ICM 225
Mines ICM 158
Mines ICM 168
Mines ICM 186
Mines ICM 113
Mines 

Mines ICM 133
Mines ICM 271
Mines ICM 320
Mines ICM 228
Mines ICM 194
Mines ICM 209
Mines ICM 216
Mines ICM 240
Mines ICM 189
Mines ICM 244
Mines ICM 250
Mines ICM 337
Mines ICM 328
Mines ICM 86
Mines ICM 276
Mines ICM 315
Mines ICM 188
Mines ICM 195
Mines ICM 189
Mines ICM 168
Mines ICM 302
Mines ICM 230
Mines ICM 316
Mines ICM 286
Mines ICM 268
Mines ICM 271
Mines ICM 314
Mines ICM 210
Mines ICM 159
Mines ICM 309
Mines ICM 298
Mines ICM 266
Mines ICM 299
Mines ICM 323
Mines ICM 267
Mines ICM 134
Mines ICM 214
Mines ICM 227
Mines ICM 268
Mines ICM 135
Mines ICM 256
Mines ICM 129
Mines ICM 410
Mines ICM 213
Mines ICM 167
Mines ICM 257
Mines ICM 308
Mines ICM 153
Mines ICM 322
Mines ICM 298
Mines ICM 323
Mines ICM 235
Mines ICM 281
Mines ICM 276
Mines ICM 312
Mines ICM 289
Mines ICM 242
Mines ICM 326
Mines ICM 170
Mines ICM 354
Mines ICM 327
Mines ICM 333
Mines ICM 261
Mines ICM 294
Mines ICM 323
Mines ICM 161
Mines ICM 362
Mines ICM 216
Mines ICM 211
Mines ICM 246
Mines ICM 215
Mines I

...

Campus 9 211
Campus 9 257
Campus 9 241
Campus 9 188
Campus 9 326
Campus 9 184
Campus 9 152
Campus 9 185
Campus 9 92
Campus 9 132
Campus 9 190
Campus 9 219
Campus 9 155
Campus 9 323
Campus 9 192
Campus 9 197
Campus 9 132
Campus 9 140
Campus 9 148
Campus 9 108
Campus 9 42
Campus 9 174
Campus 9 190
Campus 9 204
Campus 9 174
Campus 9 109
Campus 9 174
Campus 9 201
Campus 9 271
Campus 9 205
Campus 9 190
Campus 9 270
Campus 9 181
Campus 9 109
Campus 9 276
Campus 9 123
Campus 9 153
Campus 9 140
Campus 9 216
Campus 9 248
Campus 9 180
Campus 9 342
Campus 9 131
Campus 9 126
Campus 9 215
Campus 9 173
Campus 9 351
Campus 9 123
Campus 9 87
Campus 9 152
Campus 9 128
Campus 9 92
Campus 9 125
Campus 9 180
Campus 9 175
Campus 9 185
Campus 9 199
Campus 9 166
Campus 9 195
Campus 9 146
Campus 9 113
Campus 9 169
Campus 9 308
Campus 9 274
Campus 9 243
Campus 9 101
Campus 9 305
Campus 9 232
Campus 9 241
Campus 9 149
Campus 9 188
Campus 9 136
Campus 9 100
Campus 9 178
Campus 9 188
Campus 9 247
Campus 9 250
Cam

Campus 9 153
Campus 9 221
Campus 9 221
Campus 9 176
Campus 9 358
Campus 9 178
Campus 9 153
Campus 9 133
Campus 9 175
Campus 9 175
Campus 9 126
Campus 9 30
Campus 9 187
Campus 9 199
Campus 9 129
Campus 9 146
Campus 9 133
Campus 9 154
Campus 9 217
Campus 9 270
Campus 9 221
Campus 9 174
Campus 9 276
Campus 9 167
Campus 9 140
Campus 9 255
Campus 9 133
Campus 9 157
Campus 9 136
Campus 9 190
Campus 9 233
Campus 9 166
Campus 9 276
Campus 9 136
Campus 9 139
Campus 9 179
Campus 9 138
Campus 9 290
Campus 9 150
Campus 9 84
Campus 9 175
Campus 9 127
Campus 9 112
Campus 9 131
Campus 9 192
Campus 9 159
Campus 9 193
Campus 9 215
Campus 9 175
Campus 9 227
Campus 9 113
Campus 9 100
Campus 9 127
Campus 9 176
Campus 9 226
Campus 9 198
Campus 9 82
Campus 9 235
Campus 9 246
Campus 9 189
Campus 9 173
Campus 9 201
Campus 9 142
Campus 9 85
Campus 9 229
Campus 9 200
Campus 9 211
Campus 9 281
Campus 9 157
Campus 9 291
Campus 9 69
Campus 9 214
Campus 9 198
Campus 9 138
Campus 9 232
Campus 9 51
Campus 9 151
Campu

Campus 10 194
Campus 10 259
Campus 10 192
Campus 10 271
Campus 10 330
Campus 10 175
Campus 10 282
Campus 10 329
Campus 10 153
Campus 10 141
Campus 10 228
Campus 10 209
Campus 10 193
Campus 10 218
Campus 10 219
Campus 10 168
Campus 10 205
Campus 10 268
Campus 10 206
Campus 10 115
Campus 10 199
Campus 10 241
Campus 10 237
Campus 10 372
Campus 10 170
Campus 10 294
Campus 10 233
Campus 10 378
Campus 10 205
Campus 10 332
Campus 10 411
Campus 10 389
Campus 10 189
Campus 10 316
Campus 10 257
Campus 10 178
Campus 10 152
Campus 10 179
Campus 10 208
Campus 10 236
Campus 10 355
Campus 10 281
Campus 10 309
Campus 10 296
Campus 10 220
Campus 10 221
Campus 10 213
Campus 10 294
Campus 10 165
Campus 10 209
Campus 10 354
Campus 10 276
Campus 10 289
Campus 10 146
Campus 10 305
Campus 10 125
Campus 10 180
Campus 10 260
Campus 10 226
Campus 10 328
Campus 10 265
Campus 10 232
Campus 10 233
Campus 10 270
Campus 10 255
Campus 10 189
Campus 10 104
Campus 10 281
Campus 10 248
Campus 10 240
Campus 10 152
Campus

Campus 10 235
Campus 10 232
Campus 10 204
Campus 10 213
Campus 10 242
Campus 10 144
Campus 10 158
Campus 10 174
Campus 10 154
Campus 10 265
Campus 10 189
Campus 10 142
Campus 10 127
Campus 10 292
Campus 10 177
Campus 10 127
Campus 10 199
Campus 10 169
Campus 10 139
Campus 10 164
Campus 10 152
Campus 10 162
Campus 10 203
Campus 10 183
Campus 10 199
Campus 10 136
Campus 10 212
Campus 10 248
Campus 10 113
Campus 10 111
Campus 10 153
Campus 10 223
Campus 10 173
Campus 10 200
Campus 10 152
Campus 10 303
Campus 10 137
Campus 10 168
Campus 10 445
Campus 10 353
Campus 10 305
Campus 10 241
Campus 10 281
Campus 10 134
Campus 10 183
Campus 10 169
Campus 10 208
Campus 10 278
Campus 10 214
Campus 10 193
Campus 10 295
Campus 10 492
Campus 10 254
Campus 10 248
Campus 10 228
Campus 10 317
Campus 10 212
Campus 10 235
Campus 10 344
Campus 10 79
Campus 10 233
Campus 10 290
Campus 10 338
Campus 10 356
Campus 10 181
Campus 10 253
Campus 10 246
Campus 10 303
Campus 10 263
Campus 10 327
Campus 10 252
Campus 

Campus 10 221
Campus 10 321
Campus 10 304
Campus 10 425
Campus 10 313
Campus 10 140
Campus 10 375
Campus 10 367
Campus 10 254
Campus 10 263
Campus 10 352
Campus 10 130
Campus 10 216
Campus 10 281
Campus 10 165
Campus 10 265
Campus 10 419
Campus 10 230
Campus 10 229
Campus 10 253
Campus 10 370
Campus 10 248
Campus 10 290
Campus 10 234
Campus 10 144
Campus 10 131
Campus 10 217
Campus 10 137
Campus 10 148
Campus 10 237
Campus 10 147
Campus 10 255
Campus 10 238
Campus 10 239
Campus 10 274
Campus 10 267
Campus 10 286
Campus 10 249
Campus 10 366
Campus 10 291
Campus 10 255
Campus 10 195
Campus 10 276
Campus 10 316
Campus 10 174
Campus 10 130
Campus 10 339
Campus 10 122
Campus 10 248
Campus 10 88
Campus 10 322
Campus 10 400
Campus 10 234
Campus 10 332
Campus 10 250
Campus 10 437
Campus 10 328
Campus 10 184
Campus 10 298
Campus 10 200
Campus 10 370
Campus 10 268
Campus 10 319
Campus 10 267
Campus 10 194
Campus 10 131
Campus 10 153
Campus 10 231
Campus 10 241
Campus 10 224
Campus 10 223
Campus 

In [459]:
# finalList['G. Charpak']
finalList['IMT Douai']

,date,Prod_0,Prod_1,Prod_2,Prod_3,Prod_4,Prod_5,Prod_6,Prod_7,Prod_8,Prod_9,Prod_10,Prod_11,Prod_12,Prod_13,Prod_14,Prod_15
0,2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-01-02 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-01-03 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-01-04 00:00:00,73,70,81,72,66,86,91,85,73,70,71,87,73,67,96,77
4,2022-01-05 00:00:00,22,30,29,37,26,22,18,28,29,17,15,25,25,23,28,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2022-12-28 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2022-12-29 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
363,2022-12-30 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [460]:
finalList

{'G. Charpak':                     date  Prod_0  Prod_1  Prod_2  Prod_3  Prod_4
 0    2022-01-01 00:00:00       0       0       0       0       0
 1    2022-01-02 00:00:00       0       0       0       0       0
 2    2022-01-03 00:00:00       0       0       0       0       0
 3    2022-01-04 00:00:00      56      70      52      61      47
 4    2022-01-05 00:00:00      74      74      76      75      69
 ..                   ...     ...     ...     ...     ...     ...
 360  2022-12-27 00:00:00       0       0       0       0       0
 361  2022-12-28 00:00:00       0       0       0       0       0
 362  2022-12-29 00:00:00       0       0       0       0       0
 363  2022-12-30 00:00:00       0       0       0       0       0
 364  2022-12-31 00:00:00       0       0       0       0       0
 
 [365 rows x 6 columns],
 'Mines ICM':                     date  Prod_0  Prod_1  Prod_2  Prod_3  Prod_4  Prod_5  \
 0    2022-01-01 00:00:00       0       0       0       0       0       0   


In [465]:
print(type(finalList['IMT Douai']['date'][0]))

<class 'datetime.datetime'>


In [474]:
import xlsxwriter
from datetime import datetime

with pd.ExcelWriter('DailyGroupedOrders.xlsx', engine = 'xlsxwriter', datetime_format = 'dd mm yyyy') as writer:
    for camp in finalList:
        finalList[camp].to_excel(writer, sheet_name = camp)

In [ ]:
test = pd.DataFrame({'date' : intermediaire['date']})
test

In [ ]:
intermediaire

In [ ]:
for name in campus['Liste']:
    for i in range (0,intermediaire[name].shape[0]):
        print(name)
        dailyOrders = intermediaire.iloc[i][name]
        print (dailyOrders)

librairie pulp